In [1]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from tensorflow import keras
from tensorflow.keras import layers
import scipy.misc
from matplotlib.pyplot import imshow
from keras.layers import Dropout
import h5py
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline

import keras.backend as K
import numpy as np
import tensorflow as tf

In [63]:
def convolutional_module(X_seq, filter_size = 3):
    X_seq_conv = []
    
    for X in X_seq:
        print(X)
        '''Layer 1'''
        X = Conv2D(16, kernel_size = filter_size, strides = 2, name = 'conv1',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.3)(X)
        X = Activation('relu')(X)
        '''Layer 2'''
        X = Conv2D(16, kernel_size = filter_size, strides = 1, name = 'conv2', padding = 'same',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.3)(X)
        X = Activation('relu')(X)
        '''Layer 3'''
        X = Conv2D(16, kernel_size = filter_size, strides = 2, name = 'conv3',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.4)(X)
        X = Activation('relu')(X)
        '''Layer 4'''
        X = Conv2D(16, kernel_size = filter_size, strides = 1, name = 'conv4', padding = 'same',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.4)(X)
        X = Activation('relu')(X)
        '''Layer 5'''
        X = Conv2D(32, kernel_size = filter_size, strides = 2, name = 'conv5',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.5)(X)
        X = Activation('relu')(X)
        '''Layer 6'''
        X = Conv2D(32, kernel_size = filter_size, strides = 1, name = 'conv6', padding = 'same',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.5)(X)
        X = Activation('relu')(X)
        '''Layer 7'''
        X = Conv2D(32, kernel_size = filter_size, strides = 2, name = 'conv7',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.5)(X)
        X = Activation('relu')(X)  
        '''Layer 8'''
        X = Conv2D(32, kernel_size = filter_size, strides = 1, name = 'conv8', padding = 'same',
                   kernel_initializer = glorot_uniform(seed=0))(X)
        X = Dropout(0.5)(X)
        X = Activation('relu')(X)
        
        X_seq_conv.append(X)
    
    X_seq_conv = tf.stack(X_seq_conv)

    return X_seq_conv
    
    
    

## Convolutional Layers
We specified the sequence of convolution layers as follows (see Fig. 1): conv3-16, conv3-16, conv3-16, conv3-16, conv3-32, conv3-32, conv3-32, conv3-32 (notation: conv(kernel size) - (number of kernels)). Weight filters are of shape [f, f, #filters]

In [128]:
def lstm_module(X):
    return X

In [129]:
def DeepLight(input_shape, classes, padding):
    
    '''Define Input as Tensor with pre-defined shape'''
    X_input = Input(input_shape)
    
    '''Potential Zero-padding'''
    X = ZeroPadding2D((padding, padding))(X_input)
    
    '''Sequence the acial slices'''
    X = tf.unstack(X)
    
    '''Convolutional Neural Network (8 Layers), run each axial slice in ith training example sequence through the convolutional_module'''
    X = convolutional_module(X)
    
    '''Long-Short Term Memory Bi-Directional Recurrent Network (2 Units)'''
    X = lstm_module(X)
    
    '''Fully-Connected Layer (Softmax Output)'''
    X_seq = Flatten()(X)
    X = Dense(classes, activation = 'softmax', name = 'FC' + str(classes), 
              kernel_initializer = glorot_uniform(seed=0))(X)
    
    '''Initialize DeepLight Model'''
    dl_model = Model(inputs = X_input, outputs = X, name = 'DeepLight_v1')
    
    return dl_model
    

### DeepLight Training
"We iteratively trained DeepLight through backpropagation [Rumelhart et al., 1986] over a period of 60
epochs by the use of the ADAM optimization algorithm as implemented in tensorflow 1.4 [Abadi et al., 2016].
To prevent overfitting, we applied dropout regularization to all network layers [Srivastava et al., 2014],
global gradient norm clipping (with a clipping threshold of 5) [Pascanu et al., 2013], as well as an early
stopping of the training (see Supplementary Fig. S2 for an overview of training statistics). During
the training, we set the dropout probability to 50% for all network layers, except for the first four
8
convolution layers, where we reduced the dropout probability to 30% for the first two layers and 40%
for the third and fourth layer. Each training epoch was defined as a complete iteration over all samples
in the training dataset (see Section 2.2). We used a learning rate of 0.0001 and a batch size of 32.
All network weights were initialized by the use of a normal-distributed random initialization scheme"
- Analyzing Neuroimaging Data Through Recurrent Deep Learning Models, Thomas et al. (2019)


In [130]:
model = DeepLight(input_shape = [64, 64, 64], classes = 4, padding = 3)

Adam = keras.optimizers.Adam(learning_rate = 0.0001, clipnorm = 5)

model.compile(optimizer = Adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_deeplight_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


NotImplementedError: Cannot convert a symbolic Tensor (zero_padding2d_35/Pad:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [110]:
####DO NOT RUN####
'''Test at each epoch in training, the test set accuracy/loss. Stop when loss hasn't gone down in 5 epochs. 
Save the model with the best test accuracy'''
model.fit(X_train, Y_train, validation_set = [X_test, Y_test], epochs = 60, batch_size = 32, callbacks = [es, mc])

NameError: name 'X_train' is not defined